In [16]:
from keras.datasets import imdb
import numpy as np
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dense, Dropout, Activation

In [17]:
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=10000)#选取前10000个最常见的单词

In [18]:
train_data[0]#单词索引

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [19]:
train_labels[0]#0 负面 1 正面

1

In [20]:
word_index=imdb.get_word_index()#word_index将单词映射为整数索引的字典
reverse_word_index=dict([(value,key)for (key,value) in word_index.items()])#键值颠倒，将整数索引映射为单词
decoded_review=' '.join([reverse_word_index.get(i-3,'?') for i in train_data[0]])#前三个为保留索引

AttributeError: 'dict' object has no attribute 'item'

In [21]:
#把整数序列编码为二进制矩阵
def vectorize_sequences(sequences,dimension=10000):
    results=np.zeros((len(sequences),dimension))#创建一个形状为[len(sequences),dimension]的零矩阵
    for i ,sequence in enumerate(sequences):
        results[i,sequence]=1  #在train_data[i]存在的单词索引为1  单词索引也是数字
    return results

x_train= vectorize_sequences(train_data)
x_test= vectorize_sequences(test_data)

#标签向量化

y_train=np.asarray(train_labels).astype('float32')
y_test=np.asarray(test_labels).astype('float32')

In [30]:
#模型定义
model=models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(10000,)))#小于0则输出为0  
model.add(Dropout(0.5))#节点50%的丢弃率
model.add(layers.Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))

#配置优化器
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss='binary_crossentropy',#使用交叉熵
             metrics=['accuracy'])

In [31]:
#验证集
x_val=x_train[:10000]
partial_x_train=x_train[10000:]

y_val=y_train[:10000]
partial_y_train=y_train[10000:]

In [32]:
#训练模型  
history=model.fit(partial_x_train,
                 partial_y_train,
                 epochs=20,
                 batch_size=512,
                 validation_data=(x_val,y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 4s 254us/step - loss: 0.6407 - acc: 0.6139 - val_loss: 0.5494 - val_acc: 0.8046
Epoch 2/20
15000/15000 [==============================] - 3s 202us/step - loss: 0.5237 - acc: 0.7535 - val_loss: 0.4357 - val_acc: 0.8586
Epoch 3/20
15000/15000 [==============================] - 3s 203us/step - loss: 0.4380 - acc: 0.8123 - val_loss: 0.3501 - val_acc: 0.8780
Epoch 4/20
15000/15000 [==============================] - 3s 214us/step - loss: 0.3677 - acc: 0.8555 - val_loss: 0.3054 - val_acc: 0.8846
Epoch 5/20
15000/15000 [==============================] - 3s 205us/step - loss: 0.3158 - acc: 0.8819 - val_loss: 0.3045 - val_acc: 0.8740
Epoch 6/20
15000/15000 [==============================] - 3s 206us/step - loss: 0.2833 - acc: 0.8988 - val_loss: 0.2769 - val_acc: 0.8874
Epoch 7/20
15000/15000 [==============================] - 3s 205us/step - loss: 0.2501 - acc: 0.9131 - val_loss: 0.2825 - 

In [33]:
history_dict=history.history
history_dict.keys()

dict_keys(['acc', 'val_loss', 'loss', 'val_acc'])

In [34]:
loss_values=history_dict['val_loss']
val_loss_values=history_dict['val_loss']
print(loss_values)
print(val_loss_values)

[0.54944855012893679, 0.43566300020217896, 0.35009291930198672, 0.30537174491882324, 0.30452811541557312, 0.27687929182052612, 0.28247027359008792, 0.29132863259315489, 0.2955847466945648, 0.31816559724807741, 0.32115684018135071, 0.34870341534614563, 0.39229655008316039, 0.39925487208366395, 0.39955431141853331, 0.43752663850784301, 0.47845565342903135, 0.49046609005928038, 0.49803362994194028, 0.51546882958412166]
[0.54944855012893679, 0.43566300020217896, 0.35009291930198672, 0.30537174491882324, 0.30452811541557312, 0.27687929182052612, 0.28247027359008792, 0.29132863259315489, 0.2955847466945648, 0.31816559724807741, 0.32115684018135071, 0.34870341534614563, 0.39229655008316039, 0.39925487208366395, 0.39955431141853331, 0.43752663850784301, 0.47845565342903135, 0.49046609005928038, 0.49803362994194028, 0.51546882958412166]


In [36]:
results=model.evaluate(x_test,y_test)#训练
results[1]

25000/25000 [==============================] - 4s 159us/step


0.87283999999999995